In [1]:
import requests
from requests.auth import HTTPBasicAuth

from config import settings, constants
from source.map_utils import visualize_stations, visualize_pjm_stations, get_pjm_regions_for_weather_stations

In [2]:
get_pjm_regions_for_weather_stations()

TOKEN VALID UNTIL: 2024-06-21 13:54:03.369817 -- NOW: 2024-06-21 13:24:04.506500
1/1269: US1VAAB0001 VA (37.9510, -78.4630) --> PJM_DC, 0:00:01.127393
2/1269: US1VAAB0002 VA (38.1391, -78.5148) --> PJM_DC, 0:00:01.125368
3/1269: US1VAAB0006 VA (38.0131, -78.7447) --> PJM_DC, 0:00:01.230313
4/1269: US1VAAB0007 VA (38.1108, -78.5119) --> PJM_DC, 0:00:01.165903
5/1269: US1VAAB0009 VA (38.0116, -78.3751) --> PJM_DC, 0:00:01.392197
6/1269: US1VAAB0010 VA (38.0196, -78.6389) --> PJM_DC, 0:00:01.176742
7/1269: US1VAAB0012 VA (38.0453, -78.3281) --> PJM_DC, 0:00:01.141097
8/1269: US1VAAB0017 VA (38.0664, -78.7222) --> PJM_DC, 0:00:01.163276
9/1269: US1VAAB0018 VA (38.1256, -78.4312) --> PJM_DC, 0:00:01.125874
10/1269: US1VAAB0019 VA (38.0267, -78.4732) --> PJM_DC, 0:00:01.167604
11/1269: US1VAAB0021 VA (37.9355, -78.6436) --> PJM_DC, 0:00:01.060953
12/1269: US1VAAB0022 VA (38.0578, -78.4520) --> PJM_DC, 0:00:01.012117
13/1269: US1VAAB0027 VA (38.1606, -78.4704) --> PJM_DC, 0:00:01.066289
14/12

In [4]:
import os
import polars as pl
from source.utils import read_pjm_data
from config import settings, constants

first_year = 2012
last_year = 2024

year = 2012
agg_level = constants.NERC_REGION
file_path = os.path.join(settings.PJM_FOLDER, f'pjm-{year}.csv')
pjm_df = read_pjm_data(start_year=2012, end_year=2024, aggregation_level = agg_level)
raw_df = pl.read_csv(source=file_path)


In [5]:
print(f'{raw_df.columns}')
print(f'{pjm_df.columns}')

raw_df.with_columns(
    pl.col(agg_level) == 'RFC'
)

['datetime_beginning_utc', 'datetime_beginning_ept', 'nerc_region', 'mkt_region', 'zone', 'load_area', 'mw', 'is_verified']
['local_date_time', 'RFC', 'RTO', 'SERC']


datetime_beginning_utc,datetime_beginning_ept,nerc_region,mkt_region,zone,load_area,mw,is_verified
str,str,bool,str,str,str,f64,bool
"""1/1/2012 5:00:00 AM""","""1/1/2012 12:00:00 AM""",true,"""MIDATL""","""BC""","""BC""",3117.523,true
"""1/1/2012 5:00:00 AM""","""1/1/2012 12:00:00 AM""",true,"""MIDATL""","""CNCT""","""AE""",1017.566,true
"""1/1/2012 5:00:00 AM""","""1/1/2012 12:00:00 AM""",true,"""MIDATL""","""CNCT""","""DPL""",1745.034,true
"""1/1/2012 5:00:00 AM""","""1/1/2012 12:00:00 AM""",true,"""MIDATL""","""GPU""","""JC""",2171.4,true
"""1/1/2012 5:00:00 AM""","""1/1/2012 12:00:00 AM""",true,"""MIDATL""","""GPU""","""ME""",1377.857,true
…,…,…,…,…,…,…,…
"""1/1/2013 4:00:00 AM""","""12/31/2012 11:00:00 PM""",true,"""WEST""","""DAY""","""DAY""",1790.0,true
"""1/1/2013 4:00:00 AM""","""12/31/2012 11:00:00 PM""",true,"""WEST""","""DEOK""","""DEOK""",2876.0,true
"""1/1/2013 4:00:00 AM""","""12/31/2012 11:00:00 PM""",true,"""WEST""","""DUQ""","""DUQ""",1600.529,true


In [7]:
import polars as pl

df = read_pjm_data(start_year=2012, end_year=2024, settings=settings, constants=constants)

df = df.with_columns(
    pl.sum_horizontal(constants.PJM_ZONES).alias('zone sum')    
)
df = df.with_columns(
    (pl.col('zone sum') == pl.col('RTO')).alias('check'),
    (pl.col('zone sum') - pl.col('RTO')).abs().alias('diff')
)
df

local_date_time,AE,AEP,AP,ATSI,BC,CE,CNCT,DAY,DEOK,DOM,DPL,DUQ,EKPC,GPU,JC,ME,OVEC,PE,PEP,PL,PN,PS,RECO,RTO,zone sum,check,diff
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,f64
2012-01-01 00:00:00,null,13808.564,4542.336,6227.199,3117.523,9903.598,2762.6,1605.0,2533.0,9010.0,null,1453.608,null,5202.161,null,null,null,3797.91,2857.118,3717.775,null,4148.766,145.941,74833.099,74833.099,true,0.0
2012-01-01 01:00:00,null,13659.035,4410.591,5954.737,3044.761,9366.405,2669.852,1555.0,2465.0,8930.0,null,1406.893,null,4908.071,null,null,null,3645.334,2793.013,3584.587,null,3955.249,133.718,72482.246,72482.246,true,0.0
2012-01-01 02:00:00,null,13514.01,4329.339,5747.631,2957.325,9125.434,2603.265,1507.0,2364.0,8849.0,null,1359.895,null,4698.067,null,null,null,3512.677,2743.489,3480.515,null,3812.949,125.968,70730.564,70730.564,true,0.0
2012-01-01 03:00:00,null,13461.573,4303.711,5662.562,2997.503,8757.395,2613.612,1483.0,2313.0,8950.0,null,1336.589,null,4607.397,null,null,null,3453.96,2708.451,3389.294,null,3727.416,120.588,69886.051,69886.051,true,0.0
2012-01-01 04:00:00,null,13557.933,4360.452,5648.978,2999.98,8484.486,2628.217,1457.0,2279.0,9111.0,null,1318.973,null,4624.177,null,null,null,3460.863,2741.365,3461.836,null,3677.829,119.3,69931.389,69931.389,true,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-05-31 18:00:00,1277.371,14801.568,5343.351,7483.011,3477.62,11469.219,null,1969.0,3119.005,13841.86,2105.815,1554.793,1678.458,null,2916.889,1719.124,28.0,4313.934,3114.013,4395.951,1818.355,5343.394,191.026,91961.757,91961.757,true,0.0
2024-05-31 19:00:00,1245.508,14378.991,5204.156,7258.502,3356.135,11135.191,null,1904.0,2984.692,13481.297,2024.496,1492.308,1566.799,null,2753.727,1668.768,34.0,4180.876,2991.828,4279.94,1782.694,5216.931,182.557,89123.396,89123.396,false,1.4552e-11
2024-05-31 20:00:00,1199.394,14045.94,5050.034,7206.378,3262.563,10886.085,null,1846.0,2856.426,13050.055,1945.134,1464.418,1516.115,null,2625.218,1604.311,26.0,4042.42,2906.447,4179.552,1764.681,5079.385,174.368,86730.924,86730.924,true,0.0
